Get the reactome links from the Rhea database:

In [1]:
%%capture
%%bash

cd reactome/
wget https://ftp.expasy.org/databases/rhea/tsv/rhea2reactome.tsv

In [2]:
import pandas as pnd

table = pnd.read_csv('reactome/rhea2reactome.tsv', sep='\t')
table.head()

,RHEA_ID,DIRECTION,MASTER_ID,ID
0,10041,LR,10040,R-HSA-176606.3
1,10041,LR,10040,R-HSA-8953499.3
2,10117,LR,10116,R-HSA-71299.4
3,10133,LR,10132,R-HSA-159443.3
4,10165,LR,10164,R-HSA-71660.4


In [3]:
rhea_to_reactome = {}

for index, row in table.iterrows(): 
    if row['RHEA_ID'] not in rhea_to_reactome.keys(): 
        rhea_to_reactome[str(row['RHEA_ID'])] = set()
    rhea_to_reactome[str(row['RHEA_ID'])].add( row['ID'])
    

In [4]:
import copy
import pickle


def extend_with_reactome(something_to_others, rhea_to_reactome):

    something_to_others_basename = something_to_others
    
    # read something_to_others
    with open(f'mnx_dicts_R/{something_to_others}.pickle', 'rb') as handler:
        something_to_others = pickle.load(handler)


    something_to_others_reactome = copy.deepcopy(something_to_others)
    for something_id in something_to_others.keys():  # something_id = bigg or seed ID.
        something_to_others_reactome[something_id]['reactome'] = set()

        for rhea_id in something_to_others[something_id]['rhea']:
            try:  
                reactome_ids = rhea_to_reactome[rhea_id]
                for reactome_id in reactome_ids:
                    something_to_others_reactome[something_id]['reactome'].add(reactome_id)
            except:  
                continue
                
                
    # save sopmething to others extended with pubchem
    with open(f'reactome_dicts/{something_to_others_basename}.pickle', 'wb') as handle:
        pickle.dump(something_to_others_reactome, handle)
    
    
    return something_to_others_reactome
    

In [5]:

bigg_to_others =           extend_with_reactome('bigg_to_others', rhea_to_reactome)
bigg_to_others_extended =  extend_with_reactome('bigg_to_others_extended', rhea_to_reactome)
seed_to_others =           extend_with_reactome('seed_to_others', rhea_to_reactome)
seed_to_others_extended =  extend_with_reactome('seed_to_others_extended', rhea_to_reactome)

## testing area

In [6]:
bigg_to_others['GDH']

{'kegg.reaction': {'R00243'},
 'biocyc': {'GLUTAMATE-DEHYDROGENASE-RXN'},
 'seed.reaction': {'rxn00182', 'rxn19827', 'rxn30075', 'rxn34790', 'rxn34793'},
 'bigg.reaction': {'GDH',
  'GDH1',
  'GDHm',
  'GLUDxi',
  'R_GDH',
  'R_GDH1',
  'R_GDHm',
  'R_GLUDxi'},
 'rhea': {'15133', '15134', '15135', '15136'},
 'sabiork.reaction': {'755'},
 'ec-code': {'1.4.1.14', '1.4.1.2', '1.4.1.3', '1.4.1.4'},
 'metanetx.reaction': {'MNXR144972'},
 'reactome': {'R-HSA-70589.5', 'R-HSA-70600.5'}}

In [7]:

seed_to_others_extended['rxn19827']

{'kegg.reaction': {'R00243'},
 'biocyc': {'GLUTAMATE-DEHYDROGENASE-RXN'},
 'seed.reaction': {'rxn00182', 'rxn19827', 'rxn30075', 'rxn34790', 'rxn34793'},
 'bigg.reaction': {'GDH',
  'GDH1',
  'GDHm',
  'GLUDxi',
  'R_GDH',
  'R_GDH1',
  'R_GDHm',
  'R_GLUDxi'},
 'rhea': {'15133', '15134', '15135', '15136'},
 'sabiork.reaction': {'755'},
 'ec-code': {'1.4.1.14', '1.4.1.2', '1.4.1.3', '1.4.1.4'},
 'metanetx.reaction': {'MNXR144972'},
 'reactome': {'R-HSA-70589.5', 'R-HSA-70600.5'}}